How to convert a Google sheet to a static marketing website

templates google doc?

templates google sheet?

In [ ]:
var util = require('util');
var GoogleSpreadsheet = require('google-spreadsheet');

function getTemplates(doc) {
    if(typeof doc === 'string') {
        doc = new GoogleSpreadsheet(doc);
    }
    return util.promisify(doc.getInfo)()
        .then(info => {
            console.log('Loaded doc: ' + info.title + ' by ' + info.author.email);
            return info;
        })
        .then(s => s.worksheets.reduce((obj, worksheet) => {
            var layoutTitle = worksheet.title.toLowerCase()
                .replace(/\s*layout|\s*data|\s*template/ig, '')
                .split(/s+/ig).join('-');
            if( typeof obj[layoutTitle] == 'undefined' ) obj[layoutTitle] = {template: null, data: null};
            if( worksheet.title.toLowerCase().includes('data') ) obj[layoutTitle].data = worksheet;
            else obj[layoutTitle].template = worksheet;
            return obj;
        }, {}))
}
module.exports = getTemplates;


test templates google sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get templates from sheet', () => {
    
    it('should have at least one page', () => {
        return getTemplates(docsId)
            .then(t => {
                console.log(JSON.stringify(Object.keys(t).reduce((obj, cur) => {
                    obj[cur] = {};
                    obj[cur].template = ((t[cur].template || {}).title || '').substr(0, 1000);
                    obj[cur].data = (t[cur].data ? ('yes: ' + t[cur].data.rowCount) : 'no');
                    return obj;
                }, {}), null, 4))
                assert(Object.keys(t).length > 0, 'should have a page called template');
            })
    })
    
    
})


get google data sheet?

google sheet array objects?


In [ ]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true});
var util = require('util');

function getDataSheet(worksheet) {
    if(!worksheet) {
        return Promise.resolve(false);
    }
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => {
            var columns = [];
            return cells.reduce((acc, o, arr) => {
                if (o.row == 1) {
                    columns[o.col] = o.value;
                    return acc;
                }
                if (!acc[o.row - 2]) {
                    acc[o.row - 2] = {};
                }
                acc[o.row - 2][columns[o.col]] = md.render(o.value);
                return acc;
            }, []);
        })
}
module.exports = getDataSheet;


test google data sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get data from data sheet', () => {
    
    it('should process at least one data sheet', () => {
        return getTemplates(docsId)
            .then(t => {
                var key = Object.keys(t).filter(k => t[k].data)[0];
                assert(typeof key != 'undefined', 'should have at least one data sheet');
                return getDataSheet(t[key].data);
            })
            .then(data => {
                console.log(data.length);
                console.log(data);
                assert(data.length > 0, 'should have data');
            })
    })
    
    
})


google sheet layout template?

Convert a google sheet page to an HTML template



In [1]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true});
var Mustache = require('mustache');
var getDataSheet = importer.import('google sheet array objects');

// must do this up front so we can process all data
function getSections(rows, properties, templates) {
    var returnObj = {};
    var sections = rows.reduce((obj, row, i) => {
        var subsections = row.reduce((obj, c, j) => {
            var value = row[j + 1];
            if(c === ':section' && typeof templates[value] != 'undefined') {
                obj[value] = getDataSheet(templates[value].data)
                    .then(data => {
                        returnObj[value + '-data'] = data;
                        return getLayoutSheet(templates[value].template, properties, templates);
                    })
                    .then(template => returnObj[value] = template.join(''));
            }
            return obj;
        }, obj);
        return obj;
    }, {});
    return Promise.all(Object.values(sections)).then(() => returnObj);
}

function getRows(cells) {
    // turn cells in to rows, TODO: can getCells options do this for me?
    var rows = cells.reduce((arr, c) => {
        if(typeof arr[c.row - 1] == 'undefined') arr[c.row - 1] = [];
        arr[c.row - 1][c.col - 1] = c.value;
        return arr;
    }, []);
    return rows;
}

function getDataClasses(c, data) {
    // get classes from mustache vars used with supplied data
    var dataKeys = (data || []).reduce((keys, cur) => {
        // get all keys in data
        return keys.concat(Object.keys(cur));
    }, []).filter((k, h, a) => a.indexOf(k) == h);
    var dataClasses = dataKeys
        .filter(k => ('\\{\\{' + k + '\\}\\}', 'ig').match(c))
        .map(k => 'val-' + k)
        .join(' ')
    return dataClasses;
}

function getLayoutSheet(worksheet, properties, templates) {
    if(!worksheet) {
        return Promise.resolve(false);
    }
    var layoutTitle = worksheet.title.toLowerCase()
                .replace(/\s*layout|\s*data|\s*template/ig, '')
                .split(/s+/ig).join('-');
    var rows;
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => {
            // get all sub sections up front
            rows = getRows(cells);
            return getSections(rows, templates);
        })
        .then(sections => {
            // set object properties for mustache template
            var html = rows.reduce((arr, row, i) => {
                var rowsHtml = row.reduce((arr, c, j) => {
                    if(c.substr(0, 1) === ':') {
                        // use subsequent column for property values
                        var value = row[j + 1];
                        if(c.substr(1).toLowerCase() === 'section') {
                            arr[arr.length] = `
<div class="section-${value}">
${sections[value]}
</div>
`;
                        } else {
                            properties[c.substr(1)] = value;
                        }
                    } else if(j == 0 || j >= 1 && row[j - 1]
                              && row[j - 1].substr(0, 1) !== ':') {
                        var dataClasses = getDataClasses(c, data);
                        
                        // render markdown content if it is not the value for the previous property
                        arr[arr.length] = `
<div class="${dataClasses} cell-${j}">
${md.render(c)}
</div>
`;
                    }
                    return arr;
                }, []);
                
                // render mustache templates
                if(rowsHtml.length > 0) {
                    var rowTemplate = `
<div class="{{class}} col-{{colCount}}">
${rowsHtml.join('')}
</div>
`;
                    Mustache.parse(rowTemplate);
                    arr[arr.length] = Mustache.render(rowTemplate, Object.assign(properties, {
                        colCount: rowsHtml.length
                    }));
                }
                
                return arr;
            }, []);
        
            // render mustache page template
            Mustache.parse(html);
            return Mustache.render(html, Object.assign(properties, sections));
        })
}

module.exports = getLayoutSheet;

{ getLayoutSheet: [Function: getLayoutSheet],
  htmlSheetMarkup: [Function: htmlSheetMarkup] }

test google sheet template?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getLayoutSheet = importer.import('google sheet layout template');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get html from template', () => {
    
    it('should process at least one template', () => {
        var templates, key;
        return getTemplates(docsId)
            .then(t => {
                templates = t;
                key = Object.keys(templates).filter(k => templates[k].template)[0];
                return getDataSheet(templates[key].data);
            })
            .then(data => {
                assert(typeof key != 'undefined', 'should have at least one template');
                var properties = {};
                properties[key + '-data'] = data;
                return getLayoutSheet(templates[key].template, properties, templates);
            })
            .then(template => {
                console.log(template.join(''));
                assert(template.length > 0, 'should have a page called template');
            })
    })
    
    
})


output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:
var fs = require('fs');
var path = require('path');

function writeTemplate(html, properties) {
    // TODO: automatically set title if it isn't set manually
    
    var pageHtml = `
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no">
<link rel="icon" href="{{logo}}">
<meta property="og:type" content="website">
<meta property="og:title" content="{{title}}">
<title>{{title}}</title>
</head>
<body class="{{class}} rows-{{rowCount}}">
${html.join('')}
</body>
</html>`;
            Mustache.parse(pageHtml);
            return Mustache.render(pageHtml, Object.assign({}, properties, {
                rowCount: html.length
            }));

}

module.exports = writeTemplate;


test output sheet template?
